In [32]:
import pandas as pd

In [33]:
df = pd.read_csv("spotify_millsongdata.csv")

In [34]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [35]:
df.shape

(57650, 4)

In [36]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [37]:
df =df.sample(10000).drop('link', axis=1).reset_index(drop=True)

In [38]:
df.head(10)

,artist,song,text
0,Olivia Newton-John,Suddenly,She walks in and I'm suddenly a hero \r\nI'm ...
1,Kenny Rogers,Have A Little Faith In Me,When the road gets dark and you can no longer ...
2,Cher,Believe (Club 69 Phunk Dub),No matter how hard I try \r\nYou keep pushing...
3,Zeromancer,Dr. Online,Everybody online? \r\nIt's the beginning of t...
4,Christmas Songs,Es Ist Ein Ros' Entsprungen,"Es ist ein Ros' entsprungen, aus einer Wurzel ..."
5,Amy Grant,Takes A Little Time,[Chorus] \r\nIt takes a little time sometimes...
6,George Harrison,Simply Shady,Somebody brought the juicer \r\nI thought I'd...
7,Elton John,Born To Lose,Born to lose \r\nI've lived my life in vain ...
8,Vanessa Williams,Sister Moon,"Sister moon will be my guide, \r\nin your blu..."
9,Janis Joplin,A Woman Left Lonely,A woman left lonely will soon grow tired of wa...


In [39]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [40]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [41]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [44]:
similarity[0]

array([1.        , 0.00500002, 0.06203974, ..., 0.00659773, 0.01127382,
       0.01126909])

In [45]:
firstSong = df['song'][0]

'Suddenly'

In [49]:
df[df['song'] == firstSong]

,artist,song,text
0,Olivia Newton-John,Suddenly,she walk in and i 'm suddenli a hero i 'm take...


In [50]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [51]:
df.shape

(10000, 3)

In [52]:
recommendation(firstSong)

['Are You Ready',
 'Are You Ready',
 'Ready For You',
 "I'm Ready",
 "I Think I'm Ready",
 'Suspended In Gaffa',
 'Just The Lonely Talking Again',
 'Ready To Go Home',
 "Who's Grown Up Now",
 'Turned The Corner',
 'Can You Feel It',
 'Any Love',
 'Heaven Help My Heart',
 'Keep It In Motion',
 "Everybody's Everything",
 "I'm Alive",
 "Rough An' Ready",
 'Slow Motion',
 "I've Been Thinking About You",
 'No Other Love']

In [53]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))